# LLM Fine-Tuning with LoRA and QLoRA
In this notebook, we'll explore low-rank adapter (LoRA) techniques for LLM fine-tuning, using GitHub repositories from:
- [microsoft/LoRA](https://github.com/microsoft/LoRA)
- [artidoro/qlora](https://github.com/artidoro/qlora)

**Goals**:
1. Understand low-rank adaptation methods (LoRA) and how they facilitate parameter-efficient fine-tuning of large models.
2. Implement and test QLoRA, which also quantizes model weights (often to 4-bit precision) to reduce memory usage further.
3. Experiment with training hyperparameters (learning rate, batch size) and regularization strategies (dropout, weight decay, etc.) to see how they affect downstream performance.
4. Compare speed, memory footprint, and accuracy of standard full-parameter fine-tuning vs. LoRA vs. QLoRA.

We'll show how to do this in PyTorch, with [Hugging Face Transformers](https://github.com/huggingface/transformers) for easy training loops on LLMs.

# Install & Imports
First, we install any relevant libraries (if not already installed). This code will:
1. Install Hugging Face Transformers (for LLM loading & training).
2. Install bitsandbytes (if using QLoRA or 4-bit quantization).
3. Install datasets or any other library you need.

In [2]:
!pip install transformers
!pip install accelerate
!pip install datasets
!pip install bitsandbytes

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
)

import os
import numpy as np
import time

# If you are using bitsandbytes-based 4-bit quantization:
# import bitsandbytes as bnb

# For demonstration, let's just pick a small-ish model to test
# so we can see how LoRA/QLoRA steps look without massive compute.
MODEL_NAME = "facebook/galactica-125m"  # for example, or "EleutherAI/gpt-neo-125M"

# Background: LoRA & QLoRA

# **LoRA (Low-Rank Adaptation)**:
- Paper: _Hu et al. (2022) “LoRA: Low-Rank Adaptation of Large Language Models.” ICLR._
- Key idea: Freeze the original model weights, inject trainable low-rank matrices (the "adapters") into attention or MLP layers. Only the adapter weights are updated. This drastically reduces parameter count for fine-tuning, so we can fit training on a single GPU or smaller hardware.

# **QLoRA**:
- Paper: _Dettmers et al. (2023) “QLoRA: Efficient Finetuning of Quantized LLMs.”_
- Key idea: Quantize model weights (e.g., 4-bit) for forward/backward pass, and then add LoRA adapters. The base model remains in quantized form (saving memory), while the LoRA adapters (in higher precision) get trained.

Example architecture

LoRA typically wraps attention projection matrices (like `W_q`, `W_k`, `W_v`, or `W_out`) with low-rank decomposition. QLoRA applies 4-bit quantization on those same weight matrices, but leaves LoRA adapters in FP16 or BF16.

In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# Important for causal LM tasks; we want to pad on left if doing batch inference, etc.
tokenizer.pad_token = tokenizer.eos_token

# If we do standard full-precision load:
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",  # or just "cpu" or "cuda:0" depending on environment
    torch_dtype=torch.float16
)


# If we want to do a 4-bit load using bitsandbytes:
# base_model_4bit = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     load_in_4bit=True,
#     device_map="auto",
#     quantization_config=bnb.QuantizationConfig(
#         load_in_4bit=True,
#         bnb_4bit_compute_dtype=torch.float16,
#         bnb_4bit_use_double_quant=True,
#         bnb_4bit_quant_type='nf4'
#     )
# )

print("Base model loaded!")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Base model loaded!


# (Optional) Inject LoRA Adapters
We can wrap the above model with [PEFT (Parameter-Efficient Fine-Tuning)](https://github.com/huggingface/peft) or with the original LoRA code.

Below is a **PEFT**-style example (huggingface/peft) for simplicity. If you want the original LoRA or QLoRA repos, see their `train.py` scripts. The logic is mostly the same.

In [4]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["k_proj", "q_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model_lora = get_peft_model(base_model, lora_config)
print(f"LoRA Model params: {model_lora.print_trainable_parameters()}")

trainable params: 442,368 || all params: 125,472,768 || trainable%: 0.3526
LoRA Model params: None


# **QLoRA**:
If using QLoRA with bitsandbytes 4-bit quantization, we could do something like:

In [5]:
from peft import LoraConfig, get_peft_model

# lora_config = LoraConfig(
#     r=8,
#     lora_alpha=32,
#     target_modules=["query_key_value"],
#     lora_dropout=0.05,
#     bias="none",
#     task_type=TaskType.CAUSAL_LM
# )

# # base_model_4bit is the quantized model
# model_lora_4bit = get_peft_model(base_model_4bit, lora_config)
# print(model_lora_4bit.print_trainable_parameters())

# Then proceed with training, but the base weights remain 4-bit in memory, and only the LoRA adapter is stored in full precision.


# Prepare a Dataset
For a small demonstration, let's create a toy text dataset or use something from Hugging Face `datasets`.

In [6]:
from datasets import load_dataset

raw_dataset = load_dataset("wikitext", "wikitext-2-raw-v1")  # small example
print(raw_dataset)

# Let's define a data collator for causal LM
from transformers import DataCollatorForLanguageModeling

def tokenize_function(example):
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model_lora.resize_token_embeddings(len(tokenizer))
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = raw_dataset.map(tokenize_function, batched=True, num_proc=1, remove_columns=["text"])

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # for causal LM
)

train_dataset = tokenized_dataset["train"]
val_dataset   = tokenized_dataset["validation"]
test_dataset  = tokenized_dataset["test"]

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})


Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

# Fine-Tuning with LoRA or QLoRA

We'll use the Hugging Face `Trainer` to fine-tune.
# Training Arguments
Key parameters to watch for:
- `learning_rate` — might be smaller for LLM fine-tuning (e.g., 1e-4, 2e-5).
- `per_device_train_batch_size`, `per_device_eval_batch_size`.
- `gradient_accumulation_steps` (especially for bigger LLMs).
- `max_steps` or `num_train_epochs`.
- `fp16` or `bf16` if available (and stable).

If QLoRA, ensure `bitsandbytes` is installed and your GPU supports 4-bit.

In [13]:
training_args = TrainingArguments(
    output_dir="lora-finetune-checkpoints",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    logging_steps=50,
    fp16=True,            # Enable mixed precision training if possible
    report_to="none",     # or "wandb", "tensorboard", etc.
    max_grad_norm=1.0 # Clip gradients to prevent exploding gradients
)

# Set up the Trainer
trainer = Trainer(
    model=model_lora,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
# Train!

trainer.train()

print("Done Training LoRA model.")

Epoch,Training Loss,Validation Loss
1,3.029800,nan


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


Done Training LoRA model.


# Evaluate & Compare
Let's do a quick perplexity check on the validation or test set. We'll reuse the Trainer’s `evaluate()` method, which will compute the causal LM loss. Then perplexity = `exp(loss)`.


In [ ]:
eval_results = trainer.evaluate()
print(eval_results)
val_loss = eval_results["eval_loss"]
val_ppl = np.exp(val_loss)
print(f"Validation Perplexity: {val_ppl:.2f}")

# Homework Assignments

1. **QLoRA**: Load a 4-bit quantized model (via bitsandbytes), apply LoRA, and fine-tune. Compare GPU memory usage, speed, and resulting perplexity with the standard LoRA approach above.
2. **Hyperparameter Tuning**: Adjust `r`, `lora_alpha`, `lora_dropout`, or the learning rate.
3. **Regularization**: Introduce weight decay, dropout in LoRA layers, or regularization of adapter weights.